In [ ]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, Activation, MaxPooling2D, Dropout
from keras.layers.core import Flatten, Dense

Load dataset

In [ ]:
# Step 1 - Load data

trainingPath = 'Img/GoodImg/Bmp'
validationPath = "Img/GoodImg/Bmp"
testPath = "Img/GoodImg/Bmp"

training = ImageDataGenerator().flow_from_directory(trainingPath,
                                                target_size = (256, 256),
                                                batch_size = 16)
validation = ImageDataGenerator().flow_from_directory(validationPath,
                                                target_size = (256, 256),
                                                batch_size = 16)
test = ImageDataGenerator().flow_from_directory(testPath,
                                                target_size = (256, 256),
                                                batch_size = 16)

In [ ]:
# Step 2 - Build model
model = Sequential() # The alternative is to use a graph (more complex)

model.add(Conv2D(64, (3, 3), input_shape = (256, 256, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), input_shape = (256, 256, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), input_shape = (256, 256, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])